## <span style="color:grey">Initialization</span>

In [ ]:
%run "KiteConnect_AlgoBase.ipynb" 

##  <span style="color:green">Trading Strategy </span>

### Filtering stocks

In [7]:
### Filter Data
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.ui import Select
import re
import html5lib
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

def download_from_nse(symbol):     
    global driver
    
    driver.get("https://www.nseindia.com/products/content/equities/equities/eq_security.htm")
    #assert "Python" in driver.title
    elem = driver.find_element_by_name("symbol")
    elem.clear()
    elem.send_keys(symbol)

    dateRange = Select(driver.find_element_by_name("dateRange"))
    dateRange.select_by_value("week")


    dateRange = Select(driver.find_element_by_name("series"))
    dateRange.select_by_value("EQ")

    #submitBtn = driver.find_element_by_id("submitMe")
    #elem.submit()

    driver.execute_script("submitData()")

    try:
        element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, "download-data-link"))
        )
        oi_html= driver.find_element_by_id('historicalData').get_attribute('innerHTML')

        #f_states=   pd.read_html('https://simple.wikipedia.org/wiki/List_of_U.S._states') 
        stock_data = pd.read_html(oi_html, header=[0])[0]

    finally:
        driver.quit()
        
    return stock_data

In [ ]:
import pandas as pd
import html5lib

if platform.system() == "Windows":
    driver = webdriver.Chrome("./automation/chromedriver.exe")
else:
    driver = webdriver.Chrome("./automation/chromedriver")
    
    
oi_url = "https://www.nseindia.com/products/content/equities/equities/oi_spurts.htm"
driver.get(oi_url)

try:
    element = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "tab7Content"))
    )
    oi_html= driver.find_element_by_id('tab7Content').get_attribute('innerHTML')

    #f_states=   pd.read_html('https://simple.wikipedia.org/wiki/List_of_U.S._states') 
    data1 = pd.read_html(oi_html, header=[0])[0][1::]

finally:
    driver.quit()
    

temp_df = pd.DataFrame()

comp = lambda a,b: 1 if a>= b else 0

for index, row in data1.iterrows():
    symbol = row['Symbol']
    if re.findall('NIFTY',symbol):
        continue
    
    if re.findall('KPIT',symbol):
        symbol = 'BSOFT'
        continue
        
    oi_change = row['% Chg in OI']
    stock_data = download_from_nse(symbol)
    stock_data2 = stock_data['% Dly Qt toTraded Qty']
    
    mysum = comp(stock_data2.iloc[-1],stock_data2.iloc[-2])+comp(stock_data2.iloc[-2],stock_data2.iloc[-3])+comp(stock_data2.iloc[-3],stock_data2.iloc[-4])

    temp_data = [{'symbol':stock_data.iloc[-1]['Symbol'], 'oi_%_change':oi_change, 
                 'Today -3':stock_data2.iloc[-4], 'Today -2':stock_data2.iloc[-3], 'Today -1':stock_data2.iloc[-2],'sum':mysum,
                 'Today':stock_data2.iloc[-1], 'Prev Close':stock_data.iloc[-1]['Prev Close'], 'open':stock_data.iloc[-1]['Open Price'], 'close':stock_data.iloc[-1]['Close Price'], 'Turnover':stock_data.iloc[-1]['Turnover']}]
    #print(temp_data)

    df = pd.DataFrame.from_records(temp_data, index='symbol')
    #print(df)
    temp_df = temp_df.append(df)
    #break

temp_df.to_csv('data/nse_filter_data.csv')

temp_df = pd.read_csv('data/nse_filter_data.csv')

raw_data = temp_df.sort_values('Turnover')

check1 = temp_df['close']>temp_df['open']
check2 = temp_df['close']>temp_df['Prev Close']
chk = temp_df['Today -2']>temp_df['Today -3']
chk1 = temp_df['Today -1']>temp_df['Today -2']
chk2 = temp_df['Today']>temp_df['Today -1']


temp_df['oi_%_change']

long = (temp_df['Today']>60) & (temp_df['sum']>=2)& (chk2 == True)&((check1==True) | (check2==True))
short = (temp_df['Today']<=20) & (temp_df['sum']<=1)& (chk2 == False)&((check1==False) | (check2==False))

### Buy/Sell List

temp_df[long]

temp_df[short].sort_values('Turnover', ascending=False)

portfolio = temp_df[short].sort_values('Turnover', ascending=False)['symbol']
portfolioToken = portfolio.apply(EQTOKEN).tolist()

portfolio[0].iloc[-1]

### Trading Strategy

In [ ]:
class algoTrade(algoTrade_base):
   
    def __init__(s, symbol):
        logger.debug("AlgoTrade Called: "+symbol)
        super(algoTrade, s).__init__(symbol)
        
    
    # Long Strategies
    def long_indicators(s):
        temp_df = 0
        temp_df = temp_df | (REF(s.fastk,1) <=20)&(s.fastk>20)
        temp_df = temp_df | CROSSOVER(s.fastk, s.fastd)
        temp_df = temp_df | (REF(s.macd,1) <=0)&(s.macd>0)
        temp_df = temp_df | CROSSOVER(s.macd, s.macdsignal)
        temp_df = temp_df | (REF(s.rsi,1) <=50)
        #temp_df = temp_df | (s.CLOSE.shift(1) == s.min.shift(1)) & (s.CLOSE > s.OPEN)
        #temp_df = temp_df & CROSSOVER(s.BBB, s.CLOSE)
        
        return pd.DataFrame(  temp_df, columns=["buy"] )
     
    # Short Strategies
    def short_indicators(s):
        temp_df = 0
        temp_df = temp_df | (REF(s.fastk,1) >=80)&(s.fastk<80)
        temp_df = temp_df | CROSSOVER(s.fastd, s.fastk)
        temp_df = temp_df | (REF(s.macd,1) >=0)&(s.macd<0)
        temp_df = temp_df | CROSSOVER(s.macdsignal, s.macd)
        temp_df = temp_df | (REF(s.rsi,1) >=50)
        #temp_df = temp_df & CROSSOVER(s.CLOSE, s.BBT)
        return pd.DataFrame( temp_df, columns=["sell"])
    
    def long_ha(s):
        temp_df = (REF(s.haCLOSE,2) < REF(s.haOPEN,2)) & (REF(s.haCLOSE,1) < REF(s.haOPEN,1)) & (s.haCLOSE > s.haOPEN) 
        #temp_df = temp_df & (s.rsi < 40 )
        return pd.DataFrame( temp_df , columns=["buy"])
    
    def short_ha(s):
        temp_df = (REF(s.haCLOSE,2) > REF(s.haOPEN,2)) & (REF(s.haCLOSE,1) > REF(s.haOPEN,1)) & (s.haCLOSE < s.haOPEN)
        #temp_df = temp_df & (s.rsi > 60 )
        return pd.DataFrame( temp_df , columns=["sell"])

    def tradeDecision(s, price_ohlc_df):
        global blackoutEnabled

        super(algoTrade, s).tradeDecision(price_ohlc_df)
        
        #s.rsi = RSI(s.CLOSE, timeperiod=20)
        #s.min = MIN(s.CLOSE, timeperiod=30)
        #s.max = MAX(s.CLOSE, timeperiod=30)
        
        s.buy  = s.long_ha() & s.long_indicators()
        s.sell = s.short_ha() & s.short_indicators()
        
        s.postprocess()
        
        return (s.buy,s.sell)
        
if mode == "Algo":
    x = 'YESBANK'
    gc.collect()
    days=1
    portfolio = pd.DataFrame([x])
    toDate = dt.datetime(2019,5,3,15,15)
    #portfolio = pd.DataFrame(downloadlist)    
    print("\nPortfolio return: "+ '%.2f'%trade_simulator(portfolio, toDate, cd=True, plot=True)+"%")
    
    